Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [6]:
batch_size = 128
hidden_nodes = 1024
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  # Hidden Layer, Variables.
  weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))

  # RELU
  # relu = tf.nn.relu(tf.matmul(tf_train_dataset, weights_hidden) + biases_hidden)

  # Output layer
  # weights_output = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
  # biases_output = tf.Variable(tf.zeros([num_labels]))

  # Training computation, regularization
  logits = tf.matmul(tf_train_dataset, weights) + biases
  regularization = tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + 0.001 * regularization

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

 # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
    
#train
def train(num_steps, batch_size):
  with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print ("Initialized")
    for step in xrange(num_steps):
      # Pick an offset within the training data, which has been randomized.
      # Note: we could use better randomization across epochs.
      offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
      # Generate a minibatch.
      batch_data = train_dataset[offset:(offset + batch_size), :]
      batch_labels = train_labels[offset:(offset + batch_size), :]
      # Prepare a dictionary telling the session where to feed the minibatch.
      # The key of the dictionary is the placeholder node of the graph to be fed,
      # and the value is the numpy array to feed to it.
      feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
      _, l, predictions = session.run(
        [optimizer, loss, train_prediction], feed_dict=feed_dict)
      if (step % 500 == 0):
        print ("Minibatch loss at step", step, ":", l)
        print ("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
        print ("Validation accuracy: %.1f%%" % accuracy(
          valid_prediction.eval(), valid_labels))
    print ("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

train(1001, 128)

Initialized
Minibatch loss at step 0 : 21.5332
Minibatch accuracy: 9.4%
Validation accuracy: 9.8%
Minibatch loss at step 500 : 2.75477
Minibatch accuracy: 75.0%
Validation accuracy: 76.1%
Minibatch loss at step 1000 : 2.01148
Minibatch accuracy: 71.1%
Validation accuracy: 78.8%
Test accuracy: 85.6%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [7]:
train(3001, 128)

Initialized
Minibatch loss at step 0 : 17.9471
Minibatch accuracy: 14.8%
Validation accuracy: 15.3%
Minibatch loss at step 500 : 3.00625
Minibatch accuracy: 71.9%
Validation accuracy: 75.9%
Minibatch loss at step 1000 : 2.27867
Minibatch accuracy: 71.1%
Validation accuracy: 78.7%
Minibatch loss at step 1500 : 1.14053
Minibatch accuracy: 80.5%
Validation accuracy: 79.8%
Minibatch loss at step 2000 : 1.04951
Minibatch accuracy: 76.6%
Validation accuracy: 81.0%
Minibatch loss at step 2500 : 0.892188
Minibatch accuracy: 77.3%
Validation accuracy: 81.7%
Minibatch loss at step 3000 : 0.992894
Minibatch accuracy: 77.3%
Validation accuracy: 81.9%
Test accuracy: 88.1%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [8]:
def oneLayer():
    
  batch_size = 128
  hidden_nodes = 1024
  graph = tf.Graph()
  with graph.as_default():

      # Input data. For the training data, we use a placeholder that will be fed
      # at run time with a training minibatch.
      tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
      tf_valid_dataset = tf.constant(valid_dataset)
      tf_test_dataset = tf.constant(test_dataset)
      dropout_p = tf.placeholder(tf.float32)

      # Hidden Layer, Variables.
      weight1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
      bias1 = tf.Variable(tf.zeros([hidden_nodes]))

      weight_hidden = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
      bias_hidden = tf.Variable(tf.zeros([num_labels]))


      # Define computation and predictions # Train
      first_layer = tf.add(tf.matmul(tf_train_dataset, weight1), bias1)
      relu_layer = tf.nn.relu(first_layer)
      dropout_layer = tf.nn.dropout(relu_layer, dropout_p)
      second_layer = tf.add(tf.matmul(dropout_layer, weight_hidden), bias_hidden)
      loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(second_layer, tf_train_labels))
      train_prediction = tf.nn.softmax(second_layer)

      # Valid
      v_first_layer = tf.add(tf.matmul(tf_valid_dataset, weight1), bias1)
      v_relu_layer = tf.nn.relu(v_first_layer)
      v_second_layer = tf.add(tf.matmul(v_relu_layer, weight_hidden), bias_hidden)
      valid_prediction = tf.nn.softmax(v_second_layer)

        # Test
      t_first_layer = tf.add(tf.matmul(tf_test_dataset, weight1), bias1)
      t_relu_layer = tf.nn.relu(t_first_layer)
      t_second_layer = tf.add(tf.matmul(t_relu_layer, weight_hidden), bias_hidden)
      test_prediction = tf.nn.softmax(t_second_layer)

        # Optimizer.
      optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

    
#train
  num_steps = 3001
  with tf.Session(graph=graph) as session:
      tf.initialize_all_variables().run()
      print ("Initialized")
      for step in xrange(num_steps):
          # Pick an offset within the training data, which has been randomized.
          # Note: we could use better randomization across epochs.
          offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
          # Generate a minibatch.
          batch_data = train_dataset[offset:(offset + batch_size), :]
          batch_labels = train_labels[offset:(offset + batch_size), :]
          # Prepare a dictionary telling the session where to feed the minibatch.
          # The key of the dictionary is the placeholder node of the graph to be fed,
          # and the value is the numpy array to feed to it.
          feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, dropout_p : 0.3}
          _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
          if (step % 500 == 0):
            print ("Minibatch loss at step", step, ":", l)
            print ("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print ("Validation accuracy: %.1f%%" % accuracy(
              valid_prediction.eval(), valid_labels))
      print ("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

oneLayer()

Initialized
Minibatch loss at step 0 : 605.355
Minibatch accuracy: 10.2%
Validation accuracy: 10.7%
Minibatch loss at step 500 : 113.573
Minibatch accuracy: 68.0%
Validation accuracy: 80.8%
Minibatch loss at step 1000 : 111.527
Minibatch accuracy: 63.3%
Validation accuracy: 82.1%
Minibatch loss at step 1500 : 56.4602
Minibatch accuracy: 73.4%
Validation accuracy: 82.7%
Minibatch loss at step 2000 : 55.9944
Minibatch accuracy: 72.7%
Validation accuracy: 83.1%
Minibatch loss at step 2500 : 62.9504
Minibatch accuracy: 71.9%
Validation accuracy: 83.1%
Minibatch loss at step 3000 : 68.7053
Minibatch accuracy: 66.4%
Validation accuracy: 83.7%
Test accuracy: 89.8%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [10]:
def deepLayer():
  num_steps = 12001  
  batch_size = 128
  hidden_nodes = 2048
  graph = tf.Graph()
  with graph.as_default():

      # Input data. For the training data, we use a placeholder that will be fed
      # at run time with a training minibatch.
      tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
      tf_valid_dataset = tf.constant(valid_dataset)
      tf_test_dataset = tf.constant(test_dataset)
      dropout_p1 = tf.placeholder(tf.float32)
      dropout_p2 = tf.placeholder(tf.float32)

      # Hidden Layer, Variables.
      weight1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes], stddev=0.03))
      bias1 = tf.Variable(tf.zeros([hidden_nodes]))

      weight_hidden1 = tf.Variable(tf.truncated_normal([hidden_nodes, hidden_nodes], stddev=0.03))
      bias_hidden1 = tf.Variable(tf.zeros([hidden_nodes]))
      
      weight2 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels], stddev=0.03))
      bias2 = tf.Variable(tf.zeros([num_labels]))


      # Define computation and predictions # Train
      first_layer = tf.add(tf.matmul(tf_train_dataset, weight1), bias1)
      relu_layer1 = tf.nn.relu(first_layer)
      dropout_layer1 = tf.nn.dropout(relu_layer1, dropout_p1)
      l2_loss_1 = tf.nn.l2_loss(weight1) + tf.nn.l2_loss(bias1)
    
      
      second_layer = tf.add(tf.matmul(dropout_layer1, weight_hidden1), bias_hidden1)
      relu_layer2 = tf.nn.relu(second_layer)
      dropout_layer2 = tf.nn.dropout(relu_layer2, dropout_p2)
      l2_loss_2 = tf.nn.l2_loss(weight_hidden1) + tf.nn.l2_loss(bias_hidden1)
        
      out = tf.add(tf.matmul(dropout_layer2, weight2), bias2)
      l2_loss_3 = tf.nn.l2_loss(weight2) + tf.nn.l2_loss(bias2)
        
      # Regularization
      regularization = l2_loss_1 + l2_loss_2 + l2_loss_3
        
      loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(out, tf_train_labels)) 0.001 * regularization
      train_prediction = tf.nn.softmax(out)

      # Valid
      v_first_layer = tf.add(tf.matmul(tf_valid_dataset, weight1), bias1)
      v_relu_layer1 = tf.nn.relu(v_first_layer)
        
      v_second_layer = tf.add(tf.matmul(v_relu_layer1, weight_hidden1), bias_hidden1)
      v_relu_layer2 = tf.nn.relu(v_second_layer)
    
      v_out = tf.add(tf.matmul(v_relu_layer2, weight2), bias2)
      valid_prediction = tf.nn.softmax(v_out)
      

        # Test
      t_first_layer = tf.add(tf.matmul(tf_test_dataset, weight1), bias1)
      t_relu_layer1 = tf.nn.relu(t_first_layer)
        
      t_second_layer = tf.add(tf.matmul(t_relu_layer1, weight_hidden1), bias_hidden1)
      t_relu_layer2 = tf.nn.relu(t_second_layer)
        
      t_out = tf.add(tf.matmul(t_relu_layer2, weight2), bias2)
      test_prediction = tf.nn.softmax(t_out)

        # Optimizer.
      batch = tf.Variable(0)
      learning_rate = tf.train.exponential_decay(0.01,                # Base learning rate.
                      batch * batch_size,  # Current index into the dataset.
                      num_steps,          # Decay step.
                      0.95,                # Decay rate.
                      staircase=True)
        # Use simple momentum for the optimization.
      optimizer = tf.train.GradientDescentOptimizer(learning_rate,
                                     ).minimize(loss,
                                                   global_step=batch)
      #optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

    
#train
  
  with tf.Session(graph=graph) as session:
      tf.initialize_all_variables().run()
      print ("Initialized")
      for step in xrange(num_steps):
          # Pick an offset within the training data, which has been randomized.
          # Note: we could use better randomization across epochs.
          offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
          # Generate a minibatch.
          batch_data = train_dataset[offset:(offset + batch_size), :]
          batch_labels = train_labels[offset:(offset + batch_size), :]
          # Prepare a dictionary telling the session where to feed the minibatch.
          # The key of the dictionary is the placeholder node of the graph to be fed,
          # and the value is the numpy array to feed to it.
          feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, dropout_p1 : 0.75, dropout_p2: 0.9}
          _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
          if (step % 500 == 0):
            print ("Minibatch loss at step", step, ":", l)
            print ("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print ("Validation accuracy: %.1f%%" % accuracy(
              valid_prediction.eval(), valid_labels))
      print ("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

deepLayer()

Initialized
Minibatch loss at step 0 : 2.35381
Minibatch accuracy: 11.7%
Validation accuracy: 9.5%
Minibatch loss at step 500 : 0.715015
Minibatch accuracy: 79.7%
Validation accuracy: 80.8%
Minibatch loss at step 1000 : 0.783066
Minibatch accuracy: 77.3%
Validation accuracy: 82.1%
Minibatch loss at step 1500 : 0.577091
Minibatch accuracy: 85.9%
Validation accuracy: 83.0%
Minibatch loss at step 2000 : 0.646989
Minibatch accuracy: 82.8%
Validation accuracy: 83.2%
Minibatch loss at step 2500 : 0.657024
Minibatch accuracy: 81.2%
Validation accuracy: 83.5%
Minibatch loss at step 3000 : 0.75071
Minibatch accuracy: 78.1%
Validation accuracy: 83.8%
Minibatch loss at step 3500 : 0.439727
Minibatch accuracy: 85.9%
Validation accuracy: 83.8%
Minibatch loss at step 4000 : 0.522037
Minibatch accuracy: 85.9%
Validation accuracy: 84.0%
Minibatch loss at step 4500 : 0.615791
Minibatch accuracy: 78.9%
Validation accuracy: 84.0%
Minibatch loss at step 5000 : 0.584439
Minibatch accuracy: 84.4%
Validation